<a href="https://colab.research.google.com/github/atakanksha/data245_harth/blob/main/DATA245_harth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/atakanksha/data245_harth.git

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import decomposition, manifold, preprocessing, model_selection

sns.set_theme()

### Parameters

In [ ]:
test_size = 0.2
train_size = 1 - test_size
random_seed = 42

class_code_to_class_name = {
    1: 'walking',
    2: 'running',
    3: 'shuffling',
    4: 'stairs (ascending)',
    5: 'stairs (descending)',
    6: 'standing',
    7: 'sitting',
    8: 'lying',
    13: 'cycling (sit)',
    14: 'cycling (stand)',
    130: 'cycling (sit, inactive)',
    140: 'cycling (stand, inactive)',
}

class_code_to_id = {}
id_to_class_code = {}
id_to_class_name = {}
for i, code in enumerate(class_code_to_class_name.keys()):
    class_code_to_id[code] = i
    id_to_class_code[i] = code
    id_to_class_name[i] = class_code_to_class_name[code]

num_classes = len(class_code_to_id)

In [ ]:
def remap_labels(y):
    '''Converts class codes into consecutive ids.'''
    remap_y = y.copy()
    for class_code, class_id in class_code_to_id.items():
        remap_y[y == class_code] = class_id
    return remap_y

X = np.load('./data245_harth/features.npy')
y_orig = np.load('./data245_harth/labels.npy')
y = remap_labels(y_orig)

print(f'Number of classes: {num_classes}')
print(f'Number of features: {X.shape[1]}')
print(f'Number of samples: {X.shape[0]}')

In [ ]:
def plot_class_distribution(y, ax, title):
    labels, counts = np.unique(y, return_counts=True)
    counts = counts / counts.sum()

    ax.bar(labels, counts, align='center')
    ax.set_xticks(list(id_to_class_name.keys()), list(id_to_class_name.values()), rotation=45, ha='right')
    ax.set_title(title)

### Create test set

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X,
                                                                    y,
                                                                    stratify=y,
                                                                    test_size=test_size,
                                                                    random_state=random_seed)
print(f'Number of train samples: {X_train.shape[0]}')
print(f'Number of test samples: {X_test.shape[0]}')

fig = plt.figure(figsize=(15, 5), constrained_layout=True)
gs = fig.add_gridspec(1,3)

ax = fig.add_subplot(gs[0, 0])
plot_class_distribution(y, ax, title='Whole dataset')

ax = fig.add_subplot(gs[0, 1])
plot_class_distribution(y_train, ax, title='Train dataset')

ax = fig.add_subplot(gs[0,2])
plot_class_distribution(y_test, ax, title='Test dataset')

plt.suptitle('Distribution of classes')
plt.show()
plt.close()

### PCA

In [ ]:
def plot_pca(X_pca, y, n_components):
    X_list = [X_pca[:, i] for i in range(n_components)]

    if len(X_list) == 3:
        ax = plt.axes(projection='3d')
    else:
        ax = plt.axes()
    ax.scatter(*X_list, c=y)
    ax.set_title(f'Distribution of data using PCA with {n_components = }')
    plt.show()
    plt.close()

In [ ]:
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

pca = decomposition.PCA(n_components=2)
pca.fit(X_train_norm)
X_train_norm_pca = pca.transform(X_train_norm)
X_test_norm_pca = pca.transform(X_test_norm)

plot_pca(X_train_norm_pca, y_train, n_components=2)

pca = decomposition.PCA(n_components=3)
pca.fit(X_train_norm)
X_train_norm_pca = pca.transform(X_train_norm)
X_test_norm_pca = pca.transform(X_test_norm)
plot_pca(X_train_norm_pca, y_train, n_components=3)

In [ ]:
x_ticks = np.arange(pca.n_components_) + 1
plt.plot(x_ticks, pca.explained_variance_ratio_, 'o-', linewidth=2, color='blue')
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Variance Explained')
plt.show()

In [ ]:
def compute_model_metrics(model, X, y_true):
    y_pred = model.predict(X)

    print("Confusion Matrix")
    confusion_matrix = metrics.confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 8))
    ax = sns.heatmap(confusion_matrix, cmap="crest",  annot=True, fmt="d", linewidths=.1)
    xticks = [i + 0.5 for i in id_to_class_name.keys()]
    ax.set_xticks(
        xticks, list(id_to_class_name.values()), rotation=45, ha='right'
    )
    ax.set_yticks(
        xticks, list(id_to_class_name.values()), rotation=0, va='top'
    )
    ax.set_ylabel("True label")
    ax.set_xlabel("Predicted label")
    plt.show()

    report = metrics.classification_report(
        y_true, y_pred, target_names=list(id_to_class_name.values()), digits=4
    )
    print(report)
    test_acc = metrics.accuracy_score(y_true, y_pred)
    print(f'Accuracy: {test_acc:.4f}')

    test_precision = metrics.precision_score(y_true, y_pred, average='micro')
    print(f'Micro Precision: {test_precision:.4f}')

    test_recall = metrics.recall_score(y_true, y_pred, average='micro')
    print(f'Micro Recall/TPR: {test_recall:.4f}')

    test_f1 = metrics.f1_score(y_true, y_pred, average='micro')
    print(f'Micro f1-score: {test_f1:.4f}')

## XGBoost

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import accuracy_score, make_scorer
from xgboost import XGBClassifier

xgb_clf = XGBClassifier(
    num_class=num_classes,
    objective= 'multi:softmax',
    nthread=4,
    seed=42,
)

parameters = {
    'max_depth': [6, 8, 10],
    'n_estimators': [40, 50, 60],
    'learning_rate': [0.1, 0.01]
}

grid_search = GridSearchCV(
    xgb_clf, parameters, scoring=make_scorer(accuracy_score), cv=5, n_jobs=2, return_train_score=True
)
grid_search.fit(X_train_norm, y_train)
print(f'Best training accuracy with 5-fold cross validation: {grid_search.best_score_}')
print(f'Best parameters: \n{grid_search.best_params_}')

In [ ]:
best_xgb_model = grid_search.best_estimator_
y_test_pred = best_xgb_model.predict(X_test_norm)
acc = accuracy_score(y_test, y_test_pred)
print(f'Best test accuracy after grid search: {acc:.4f}')

In [ ]:
xgb_clf = XGBClassifier(
    num_class=num_classes,
    objective= 'multi:softmax',
    learning_rate=0.1,
    n_estimators=60,
    max_depth=8,
    nthread=4,
    seed=42
)

xgb_clf.fit(X_train_norm, y_train)

In [ ]:
compute_model_metrics(xgb_clf, X_test_norm, y_test)

## SVM

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import accuracy_score, make_scorer

# Normalize the features.
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

In [ ]:
# Grid search.
parameters = [
    {"kernel": ["rbf"], "gamma": [1e-2, 1e-3, 1e-4], "C": [10, 100, 1000]},
    {"kernel": ["linear"], "C": [1, 10]},
]

grid_search = GridSearchCV(
    SVC(), parameters, scoring=make_scorer(accuracy_score), cv=5, n_jobs=2, return_train_score=True
)
grid_search.fit(X_train_norm, y_train)
print(f'Best training accuracy with 5-fold cross validation: {grid_search.best_score_}')
print(f'Best parameters: \n{grid_search.best_params_}')

In [ ]:
best_svc_model = grid_search.best_estimator_
y_test_pred = best_svc_model.predict(X_test_norm)
acc = accuracy_score(y_test, y_test_pred)
print(f'Best test accuracy after grid search: {acc:.4f}')

In [ ]:
# Fit using the best model.
svm_clf = SVC(C=100, gamma=0.001, kernel='rbf')
svm_clf.fit(X_train_norm, y_train)

In [ ]:
compute_model_metrics(svm_clf, X_test_norm, y_test)

## KNN

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
parameters = [
    {'penalty': ['l2'], 'multi_class': ['multinomial'], 'solver':['lbfgs']},
    {'penalty': ['l2'], 'multi_class': ['ovr'], 'solver':['liblinear']},
]

grid_search = GridSearchCV(
    LogisticRegression(max_iter=100), parameters, scoring=make_scorer(accuracy_score), cv=5, n_jobs=2, return_train_score=True
)
grid_search.fit(X_train_norm, y_train)
print(f'Best training accuracy with 5-fold cross validation: {grid_search.best_score_}')
print(f'Best parameters: \n{grid_search.best_params_}')

In [ ]:
# Fit using the best model.

lr_clf = LogisticRegression(penalty="l2", multi_class='multinomial', C=10, max_iter=100)
lr_clf.fit(X_train_norm, y_train)

In [ ]:
compute_model_metrics(lr_clf, X_test_norm, y_test)

## Random Forest

## Performance with dimensionality reduction

In [ ]:
def test_with_pca_components(model, X_train, y_train, X_test, y_test, components_list):
    scaler = preprocessing.StandardScaler()
    scaler.fit(X)
    X_train_norm = scaler.transform(X_train)
    X_test_norm = scaler.transform(X_test)

    pca = decomposition.PCA()
    pca.fit(X_train_norm)
    X_train_norm_pca = pca.transform(X_train_norm)
    X_test_norm_pca = pca.transform(X_test_norm)

    accuracies = []
    for n in components_list:
        model.fit(X_train_norm_pca[:, :n], y_train)
        y_test_pred = model.predict(X_test_norm_pca[:, :n])
        acc = metrics.accuracy_score(y_test, y_test_pred)
        accuracies.append(acc)
        print(f'Testing for {n = }, {acc = }')

    return accuracies

In [ ]:
models = {
    'svm': SVC(C=100, gamma=0.001, kernel='rbf'),
    'knn': KNeighborsClassifier(n_neighbors=5),
    'logistic_regression': LogisticRegression(
        penalty="l2",
        multi_class='multinomial',
        C=10,
        max_iter=100
    ),
    'xgboost': XGBClassifier(
        num_class=num_classes,
        objective= 'multi:softmax',
        learning_rate=0.1,
        n_estimators=60,
        max_depth=8,
        nthread=4,
        seed=42
    )
}

components = [1, 2, 4, 10, 20, 40, 60, X_train.shape[1]]

accuracies = {}
for name, model in models.items():
    print(f'Running {name}')
    accuracies[name] = test_with_pca_components(model, X_train, y_train, X_test, y_test, components)

In [ ]:
plt.figure()
for name, acc in accuracies.items():
    max_acc = max(acc)
    plt.plot(components, acc, label=f'{name}: best acc = {max_acc: .4f}')

plt.xlabel('Number of PCA components')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Model accuracy as a function of number of PCA components')
plt.show()

* XGBoost's performance degrades with PCA. It achieves 94.86 % accuracy with raw features as compared to 93.56 % with PCA features.

## TSNE

In [ ]:
def plot_tsne(X, y, n_components):
    scaler = preprocessing.StandardScaler()
    X = scaler.fit_transform(X)

    print('Running TSNE')
    tsne = manifold.TSNE(
        n_components=n_components,
        init='pca',
        learning_rate='auto',
        n_jobs=4,
        random_state=0,
    )
    X_tsne = tsne.fit_transform(X)
    X_list = [X_tsne[:, i] for i in range(n_components)]

    if len(X_list) == 3:
        ax = plt.axes(projection='3d')
    else:
        ax = plt.axes()
    ax.scatter(*X_list, c=y)
    ax.set_title(f'Distribution of data using TSNE with {n_components = }')
    plt.show()
    plt.close()

plot_tsne(X_train, y_train, n_components=2)
plot_tsne(X_train, y_train, n_components=3)